# English and Spanish used cars sales description using SK 

## Load the required .NET packages and supporting classes

In [12]:
#r "nuget: Microsoft.SemanticKernel, 0.19.230804.2-preview"
#r "nuget: Microsoft.Extensions.Logging.Console, 7.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite, 7.0.9"
#r "nuget: dotenv.net, 3.1.2"

using System.ComponentModel;
using System.Net.Http;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using dotenv.net;

#!import Utils/ConsoleLogger.cs
#!import Models/Models.cs

// Json Options
readonly JsonSerializerOptions s_jsonOptions = new() { WriteIndented = true };

Installed Packages dotenv.net, 3.1.2 Microsoft.EntityFrameworkCore.Sqlite, 7.0.9 Microsoft.Extensions.Logging.Console, 7.0.0 Microsoft.SemanticKernel, 0.19.230804.2-preview

## Load the OpenAI endpoint and API key from environment variables or an .env file

In [11]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: gpt at: https://alemorfr.openai.azure.com/ with key 32c87...


## Mock an DB/API call to get the a list of cars

In [13]:
var mockCarData = new List<Car>{
    new Car("123","Ford", "Explorer", 2015, "Silver", "V6", "Platinum", 10000, 16500),
    new Car("456","Ford", "Mustang", 2018, "Blue", "V8", "Sports", 10000, 250000),
    new Car("789","Ford", "Escape", 2020, "Red", "V6","Special", 3000, 15000)
  };
  
List<Car> MockDBCall() {  
  return mockCarData;
}

## Function to get a short car description from the car properties

In [14]:
string GetCarDescription(Car car) {
  return $"{car.Year} {car.Make} {car.Model} {car.Color} {car.Motor} {car.Package} with {car.milage} miles for ${car.price}";
}

## Create an instance of a kernel

In [16]:
var kernel = new KernelBuilder()
    .WithLogger(ConsoleLogger.Log)
    .WithAzureChatCompletionService(deploymentName,endpoint, apiKey)
    .Build();

## Create the sales definition and translation in-line functions and add them to the kernel

In [17]:
// Define a function to get a used car sales description
const string fnSalesDefinition = @"Get a used car sales description for the following: 
{{$input}}.";

var salesDescriptionFunc = kernel.CreateSemanticFunction(fnSalesDefinition, 
    maxTokens: 200, temperature: 0.5, topP: 1);

// Define a function to translate English to Spanish
const string fnESTranslatorDefinition = @"Translate the following English text to Spanish: 
{{$input}}.";

var esTranslatorFunc = kernel.CreateSemanticFunction(fnESTranslatorDefinition, 
maxTokens: 200, temperature: 0.5, topP: 1);

## Using the kernel and sk functions, get a car sales description and translate it into Spanish

In [18]:
// Mock making a call to a database to get a list of cars
var cars = MockDBCall();
var text = kernel.ImportSkill(new TextSkill());
var carDesc = GetCarDescription(cars[0]);

## Get a sales description, translate it and Uppercase it in a pipe

In [19]:
var pipeLineResult = await kernel.RunAsync(carDesc,
  salesDescriptionFunc,
  esTranslatorFunc,
  text["Uppercase"]);

Console.WriteLine(pipeLineResult);

¿BUSCAS UN SUV ESPACIOSO Y LUJOSO QUE HAYA SIDO BIEN MANTENIDO Y NO TE CUESTE UN OJO DE LA CARA? ¡NO BUSQUES MÁS ALLÁ DE ESTE IMPRESIONANTE FORD EXPLORER PLATINUM 2015! CON UN ELEGANTE EXTERIOR PLATEADO QUE SEGURAMENTE LLAMARÁ LA ATENCIÓN, ESTE EXPLORER ESTÁ IMPULSADO POR UN CONFIABLE MOTOR V6 QUE OFRECE UN RENDIMIENTO SUAVE Y RECEPTIVO. Y CON SOLO 10,000 MILLAS EN EL ODÓMETRO, ¡ESTE VEHÍCULO ES PRÁCTICAMENTE NUEVO!

AL ENTRAR EN LA CABINA, TE RECIBIRÁ UN INTERIOR ESPACIOSO Y CÓMODO QUE ESTÁ CARGADO DE CARACTERÍSTICAS. EL NIVEL DE EQUIPAMIENTO PLATINUM SIGNIFICA QUE ESTE EXPLORER VIENE CON TODAS LAS CAMPANAS Y SILBATOS, INCLUYENDO ASIENTOS DE CUERO, UN SISTEMA DE SONIDO PREMIUM Y UN TECHO SOLAR PANORÁMICO. Y CON MUCHO ESPACIO DE CARGA, ESTE SUV ES PERFECTO PARA FAMIL


## Sample execute concurrent tasks

In [ ]:
// Run tasks asynchronously
// var task1 = kernel.RunAsync(carDesc,salesDescriptionFunc);
// var task2 = kernel.RunAsync(carDesc,salesDescriptionFunc);
// await Task.WhenAll(task1,task2);
// Console.WriteLine(task1.Result + " " + task2.Result);

## Get a sales description and a translation for all the cars in the mock database

In [22]:
// Keep a list of processed cars
var processedCars = new List<CarTranslation>();

foreach(var car in cars) {

  // Get a short description given the car properties
  var desc = GetCarDescription(car);
  Console.WriteLine($"Processing: {desc}");
  
  // Use the SK sales description function to get a sales for a car
  var result = await kernel.RunAsync(desc,salesDescriptionFunc);
  Console.WriteLine($"Sales Description: {result}");

  // Get the usage for the call
  Console.WriteLine(JsonSerializer.Serialize(result.ModelResults.LastOrDefault()?.GetOpenAIChatResult()?.Usage, s_jsonOptions));
  
  // Use the SK translation function to translate the sales description into Spanish
  var es = await kernel.RunAsync(result.ToString(), esTranslatorFunc);
  Console.WriteLine($"Translation: {es}");
  
  // Add processed the EN and ES sales description
  processedCars.Add(new CarTranslation(car.VIN, result.ToString(), es.ToString()));
}

// Show the processed cars
processedCars

Processing: 2015 Ford Explorer Silver V6 Platinum with 10000 miles for $16500
Sales Description: Looking for a reliable and stylish SUV that won't break the bank? Look no further than this 2015 Ford Explorer Platinum! This sleek silver vehicle comes equipped with a powerful V6 engine that will give you the performance you need, whether you're cruising down the highway or navigating city streets. With only 10000 miles on the odometer, this Explorer has plenty of life left in it and is ready to take you on all of your adventures. And with a price tag of just $16500, you won't find a better deal on a vehicle of this quality. Don't wait - come test drive this 2015 Ford Explorer Platinum today!
{
  "CompletionTokens": 132,
  "PromptTokens": 49,
  "TotalTokens": 181
}
Translation: ¿Estás buscando un SUV confiable y elegante que no te haga gastar una fortuna? ¡No busques más allá de este Ford Explorer Platinum 2015! Este elegante vehículo plateado viene equipado con un potente motor V6 que te

index value 0 CarTranslation { VIN = 123, En = Looking for a reliable and stylish SUV that won't break the bank? Look no further than this 2015 Ford Explorer Platinum! This sleek silver vehicle comes equipped with a powerful V6 engine that will give you the performance you need, whether you're cruising down the h... VIN 123 En Looking for a reliable and stylish SUV that won't break the bank? Look no further than this 2015 Ford Explorer Platinum! This sleek silver vehicle comes equipped with a powerful V6 engine that will give you the performance you need, whether you're cruising down the highway or navigating city streets. With only 10000 miles on the odometer, this Explorer has plenty of life left in it and is ready to take you on all of your adventures. And with a price tag of just $16500, you won't find a better deal on a vehicle of this quality. Don't wait - come test drive this 2015 Ford Explorer Platinum today! ES ¿Estás buscando un SUV confiable y elegante que no te haga gastar una fortuna? ¡No busques más allá de este Ford Explorer Platinum 2015! Este elegante vehículo plateado viene equipado con un potente motor V6 que te dará el rendimiento que necesitas, ya sea que estés conduciendo por la autopista o navegando por las calles de la ciudad. Con solo 10000 millas en el odómetro, este Explorer tiene mucha vida por delante y está listo para llevarte en todas tus aventuras. Y con un precio de solo $16500, no encontrarás una mejor oferta en un vehículo de esta calidad. No esperes más, ven a probar este Ford Explorer Platinum 2015 hoy mismo. 1 CarTranslation { VIN = 456, En = Looking for a thrilling ride? Look no further than this stunning 2018 Ford Mustang V8 Sports! With only 10000 miles on the clock, this car is practically brand new. The sleek blue exterior is sure to turn heads, and the powerful V8 engine under the hood will make you... VIN 456 En Looking for a thrilling ride? Look no further than this stunning 2018 Ford Mustang V8 Sports! With only 10000 miles on the clock, this car is practically brand new. The sleek blue exterior is sure to turn heads, and the powerful V8 engine under the hood will make your heart race. This Mustang is loaded with features, including a premium sound system, leather seats, and advanced safety technology. Whether you're cruising down the highway or tearing up the track, this car delivers a driving experience like no other. Don't miss your chance to own this incredible machine - contact us today to schedule a test drive! All of this can be yours for the unbeatable price of $250000. ES ¿Buscas una experiencia emocionante? ¡No busques más allá de este impresionante Ford Mustang V8 Sports del 2018! Con solo 10000 millas en el odómetro, este automóvil es prácticamente nuevo. El elegante exterior azul seguramente llamará la atención, y el poderoso motor V8 debajo del capó acelerará tu corazón. Este Mustang está equipado con características impresionantes, que incluyen un sistema de sonido premium, asientos de cuero y tecnología avanzada de seguridad. Ya sea que estés conduciendo por la autopista o corriendo en la pista, este automóvil ofrece una experiencia de conducción incomparable. ¡No pierdas la oportunidad de ser dueño de esta increíble máquina - contáctanos hoy mismo para programar una prueba de manejo! Todo esto puede ser tuyo por el precio im 2 CarTranslation { VIN = 789, En = Looking for a stylish and reliable SUV that won't break the bank? Look no further than this 2020 Ford Escape Red V6 Special! With only 3000 miles on the odometer, this vehicle is practically brand new and ready to hit the road. The powerful V6 engine provides plenty ... VIN 789 En Looking for a stylish and reliable SUV that won't break the bank? Look no further than this 2020 Ford Escape Red V6 Special! With only 3000 miles on the odometer, this vehicle is practically brand new and ready to hit the road. The powerful V6 engine provides plenty of get-up-and-go, while the sleek red exterior is sure to turn head